[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://githubtocolab.com/giswqs/gee-tutorials/blob/master/ImageCollection/reducing_image_collection.ipynb)

**Reducing an ImageCollection**

To composite images in an `ImageCollection`, use `imageCollection.reduce()`. This will composite all the images in the collection to a single image representing, for example, the min, max, mean or standard deviation of the images. (See the Reducers section for more information about reducers). For example, to create a median value image from a collection:

## Create an interactive map

In [ ]:
import ee
import geemap

In [ ]:
Map = geemap.Map()
Map

## Get Utah boundary

In [ ]:
states = ee.FeatureCollection('TIGER/2018/States')
# Map.addLayer(states, {}, "US States")

UT = states.filter(ee.Filter.eq("NAME", "Utah"))
Map.addLayer(UT, {}, "Utah", False)

buffer = ee.Feature(UT.first()).buffer(16090)
Map.addLayer(buffer, {}, "Buffer")

Map


## Load Climate data from PRISM

In [ ]:
# Load a PRISM climate normals collection
collection = ee.ImageCollection("OREGONSTATE/PRISM/Norm81m") \
    .filterBounds(buffer.geometry())
#     .filterDate('2020-01-01', '2021-01-01')

image = collection.toBands()

In [ ]:
# print(collection.getInfo())
# print(collection.first().bandNames())
print(image.getInfo())

## Filter Down to Precip

In [ ]:
# Select precipitation
precip = collection.select('ppt')

precipVis = {
  'min': 0.0,
  'max': 300.0,
  'palette': ['red', 'yellow', 'green', 'cyan', 'purple'],
}

# vis_param = {'min': 0, 
#              'max': 2000, 
#              'bands': ['B5', 'B4', 'B3'], 
#              'gamma': 1.5}

Map.setCenter(-111.421833, 39.818695, 6)
Map.addLayer(precip, precipVis, 'Precipitation')
Map.addLayer(image, {}, 'Image')

Map

In [ ]:
vis_params = {'bands': ['01_ppt'], 'min': 0.0, 'max': 100.0, 'opacity': 1.0, 'gamma': 2.32}

In [ ]:


# precip.getInfo('properties')

In [ ]:
collection.aggregate_array("CLOUD_COVER").getInfo()

In [ ]:
image = collection.first()
Map.addLayer(image, {'bands': ['B4',  'B3',  'B2'], 'max': 0.3}, 'First Image')

In [ ]:
second = ee.Image(collection.toList(collection.size()).get(1))
Map.addLayer(second, {'bands': ['B4',  'B3',  'B2'], 'max': 0.3}, 'Second Image')

## Use median reducer

At each location in the output image, in each band, the pixel value is the median of all unmasked pixels in the input imagery (the images in the collection). In the previous example, `median()` is a convenience method for the following call:

In [ ]:
# Reduce the collection with a median reducer.
median = collection.reduce(ee.Reducer.median())

# Display the median image - reducer may changes the band names
Map.addLayer(median,
             {'bands': ['B4_median',  'B3_median',  'B2_median'], 'max': 0.3},
             'also median')
Map

In [ ]:
dataset_lrf = ee.FeatureCollection('TIGER/2010/ZCTA5')
Map.addLayer(dataset_lrf, {}, "TIGER/2010/ZCTA5")

In [ ]:
median.bandNames().getInfo()

## Create an image composite

In [ ]:
collection = ee.ImageCollection('LANDSAT/LC08/C01/T1_TOA') \
    .filterBounds(UT) \
    .filterDate('2020-01-01', '2021-01-01')

In [ ]:
collection.size().getInfo()

In [ ]:
image = collection.median().clip(UT)
Map.addLayer(image, {'bands': ['B4',  'B3',  'B2'], 'max': 0.3}, 'Landsat 2020')
Map